In [16]:
!pip install eticas-audit


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [17]:
sensitive_attributes = {'sex': {'columns': [
                                            {
                                                "name": "sex",
                                                "underprivileged": [2]
                                            }
                                        ],
                                'type': 'simple'},
                        'ethnicity': {'columns': [
                                                 {
                                                     "name": "ethnicity",
                                                     "privileged": [1]
                                                 }
                                        ],
                                      'type': 'simple'},
                        'age': {'columns': [
                                            {
                                                 "name": "age",
                                                 "privileged": [3, 4]
                                            }
                                        ],
                                'type': 'simple'},
                        'sex_ethnicity': {'groups': [
                                                    "sex", "ethnicity"
                                                    ],
                                          'type': 'complex'}}



In [18]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s] %(name)s - %(message)s'
)

from eticas.model.ml_model import MLModel
model = MLModel(
    model_name="ML Testing Regression",
    description="A logistic regression model to illustrate audits",
    country="USA",
    state="CA",
    sensitive_attributes=sensitive_attributes,
    features=["feature_0", "feature_1", "feature_2"]
)



## Example Files.

### Binary: 0 or 1.

Train: files/example_training_binary_2.csv

Production: files/example_operational_binary_2.csv

Impact: files/example_impact_binary_2.csv

### Scoring: Regression from 0 to 1.

Train: files/example_training_scoring.csv

Production: files/example_operational_scoring.csv

Impact: files/example_impact_scoring.csv

In [19]:
## Download Files from repository

import requests
import os

files = ['files/example_training_binary_2.csv','files/example_operational_binary_2.csv','files/example_impact_binary_2.csv',
         'files/example_training_scoring.csv','files/example_operational_scoring.csv', 'files/example_impact_scoring.csv']

for file in files:
    url = "https://raw.githubusercontent.com/eticasai/eticas-audit/main/"+file

    carpeta_local = file.split('/')[0]
    os.makedirs(carpeta_local, exist_ok=True)  # Crea la carpeta si no existe

    # Nombre del archivo en local
    nombre_archivo_local = file.split('/')[1]
    ruta_descarga = os.path.join(carpeta_local, nombre_archivo_local)

    # Descarga
    response = requests.get(url)
    if response.status_code == 200:
        with open(ruta_descarga, "wb") as f:
            f.write(response.content)
        print("Archivo descargado en:", ruta_descarga)
    else:
        print("Error al descargar el archivo. Código de estado:", response.status_code)

Archivo descargado en: files/example_training_binary_2.csv
Archivo descargado en: files/example_operational_binary_2.csv
Archivo descargado en: files/example_impact_binary_2.csv
Archivo descargado en: files/example_training_scoring.csv
Archivo descargado en: files/example_operational_scoring.csv
Archivo descargado en: files/example_impact_scoring.csv


In [20]:
labeled_data = 'files/example_training_binary_2.csv'

production_data = 'files/example_operational_binary_2.csv'

impact_data = 'files/example_impact_binary_2.csv'

In [23]:

model.run_labeled_audit(dataset_path =labeled_data, 
                                                label_column = 'outcome', output_column = 'predicted_outcome',positive_output = [1])
model.run_production_audit(dataset_path =production_data, 
                                                output_column = 'predicted_outcome',positive_output = [1])
model.run_impacted_audit(dataset_path =impact_data, 
                                                output_column = 'recorded_outcome',positive_output = [1])
model.run_drift_audit(dataset_path_dev = labeled_data, 
                      output_column_dev = 'outcome', positive_output_dev = [1],
                    dataset_path_prod = production_data, 
                      output_column_prod = 'predicted_outcome', positive_output_prod = [1])

[INFO] eticas.model.ml_model - Running labeled audit for model: ML Testing Regression
[INFO] eticas.audit.labeled_audit - Running labeled audit on model 'ML Testing Regression'
[INFO] eticas.data.loaders - Dataset successfully loaded from files/example_training_binary_2.csv (CSV).
[INFO] eticas.audit.labeled_audit - labeled data loaded '(10000, 10)'
[INFO] eticas.metrics.da_inconsistency - Completed: 'Da_inconsistency'
[INFO] eticas.metrics.da_positive - Completed: 'Da_positive'
[INFO] eticas.metrics.dxa_inconsistency - Completed: 'Dxa_inconsistency'
[INFO] eticas.metrics.da_informative - Completed: 'Da_informative'
[INFO] eticas.metrics.da_fairness - Completed: 'Da_fairness'
[INFO] eticas.metrics.d_statisticalparity - Completed: 'D_statisticalparity'
[INFO] eticas.metrics.d_parity - Completed: 'D_parity'
[INFO] eticas.metrics.performance - Completed: 'Performance'
[INFO] eticas.metrics.d_equalodds - Completed: 'D_equalodds'
[INFO] eticas.metrics.d_calibrated - Completed: 'D_calibrated

In [24]:
import pandas as pd
pd.set_option('display.max_rows', 500)

audit_result = model.df_results(norm_values=True)

In [25]:
result = audit_result.xs(('fairness'), level=(0))
result = result.reset_index()
result = result.pivot(index=['metric','attribute'], 
                      columns='stage', 
                      values='value')
result

stage                             01-labeled 02-production 03-impact
metric              attribute                                       
d_equality          age                 99.0          99.0     100.0
                    ethnicity          100.0          99.0      44.0
                    sex                100.0          71.0      47.0
                    sex_ethnicity       99.0          62.0      44.0
d_equity            age                100.0          99.0     100.0
                    ethnicity          100.0          70.0      44.0
                    sex                 99.0          48.0      41.0
                    sex_ethnicity      100.0          57.0      44.0
d_parity            age                 98.0          98.0      99.0
                    ethnicity           99.0     76.666667      42.0
                    sex                 99.0     64.666667      40.8
                    sex_ethnicity       99.0     65.333333      40.8
d_statisticalparity age                 98.0          98.0      98.0
                    ethnicity          100.0          74.0      43.5
                    sex                100.0          57.0      42.0
                    sex_ethnicity       98.0         57.75      42.0

In [26]:
result = audit_result.xs(('distribution'), level=(0))
result = result.reset_index()
result = result.pivot(index=['metric','attribute'], 
                      columns='stage', 
                      values='value')
result

stage                           01-labeled 02-production 03-impact
metric            attribute                                       
da_informative    age                  100           100       100
                  ethnicity            100           100   72.3467
                  sex              99.3294       90.9812   60.8824
                  sex_ethnicity        100       90.9812   60.8824
dxa_inconsistency age                99.28        98.944    98.736
                  ethnicity          99.64        89.776   77.8347
                  sex               99.128       77.3333   70.1013
                  sex_ethnicity     98.808        86.264   77.8347

In [27]:
model.labeled_results['da_informative']

{'sex': {'accuracy_base': 0.507,
  'accuracy_group': 0.5087,
  'accuracy': 1.003353057199211,
  'normalized_risk': 99.3294,
  'bias_level': 'Low/no bias'},
 'ethnicity': {'accuracy_base': 0.507,
  'accuracy_group': 0.4973,
  'accuracy': 0.9808678500986193,
  'normalized_risk': 100,
  'bias_level': 'Low/no bias'},
 'age': {'accuracy_base': 0.507,
  'accuracy_group': 0.506,
  'accuracy': 0.9980276134122288,
  'normalized_risk': 100,
  'bias_level': 'Low/no bias'},
 'sex_ethnicity': {'accuracy_base': 0.507,
  'accuracy_group': 0.5007,
  'accuracy': 0.9875739644970415,
  'normalized_risk': 100,
  'bias_level': 'Low/no bias'}}